In [20]:
import pandas as pd 
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error
from math import sqrt

# Importing CSV Files

In [2]:
df_BTC = pd.read_csv("BTC_Historical_11-03.csv", index_col=0)
df_DOGE = pd.read_csv("DOGE_Historical_11-03.csv", index_col=0)
df_XRP = pd.read_csv("XRP_Historical_11-03.csv", index_col=0)
df_ETH = pd.read_csv("ETH_Historical_11-03.csv", index_col=0)
df_UNI = pd.read_csv("UNI_Historical_11-03.csv", index_col=0)

# Using feature engineering

In [ ]:
# BTC

In [9]:
df_BTC['Difference between opening and closing'] = df_BTC['Opening Price (USD)'] - df_BTC['Closing Price (USD)']

# Define features and target variable
# Assuming you want to predict the future highest price
X = df_BTC.drop(columns=['Highest Price (USD)'])
y = df_BTC['Highest Price (USD)']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train Random Forest model
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train_scaled, y_train)

# Predict on the testing set
y_pred = model.predict(X_test_scaled)

# Evaluate model performance
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

Mean Squared Error: 271535.7147975525


In [15]:
X = df_BTC.drop(columns=['Highest Price (USD)'])
y = df_BTC['Highest Price (USD)']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define a pipeline for preprocessing and modeling
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('regressor', RandomForestRegressor(random_state=42))
])

# Define hyperparameters to tune
param_grid = {
    'regressor__n_estimators': [100, 200, 300],
    'regressor__max_depth': [None, 10, 20],
    'regressor__min_samples_split': [2, 5, 10]
}

# Perform grid search cross-validation to find the best hyperparameters
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)

# Get the best model from the grid search
best_model = grid_search.best_estimator_

# Evaluate the best model on the testing set
y_pred = best_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)


Mean Squared Error: 265707.5387310729


In [19]:
train_size = int(len(df_BTC) * 0.8)
train_data, test_data = df_BTC.iloc[:train_size], df_BTC.iloc[train_size:]

# Define the SARIMA model
order = (1, 1, 1)  # ARIMA order
seasonal_order = (1, 1, 1, 12)  # SARIMA seasonal order
sarima_model = SARIMAX(train_data['Highest Price (USD)'], order=order, seasonal_order=seasonal_order, enforce_stationarity=False)

# Fit the SARIMA model
sarima_result = sarima_model.fit()

# Forecast future values
forecast_periods = 7  # Forecasting for the next 7 days
forecast = sarima_result.forecast(steps=forecast_periods)

# Create a range of future dates for the forecasted periods
future_dates = pd.date_range(start=df_BTC.index[-1], periods=forecast_periods + 1, freq='D')[1:]  # Start from the day after the last date in the dataset

# Print the forecasted values for each future date
for date, prediction in zip(future_dates, forecast):
    print(f"Predicted Highest Price for {date}: {prediction}")

/Users/nikolasstahl/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/Users/nikolasstahl/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
 This problem is unconstrained.


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            5     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  8.20122D+00    |proj g|=  3.47666D-02

At iterate    5    f=  8.19952D+00    |proj g|=  3.89565D-03

At iterate   10    f=  8.19946D+00    |proj g|=  2.42206D-03

At iterate   15    f=  8.19944D+00    |proj g|=  5.82916D-04

At iterate   20    f=  8.19923D+00    |proj g|=  8.00249D-03

At iterate   25    f=  8.17390D+00    |proj g|=  1.16124D-01

At iterate   30    f=  8.12471D+00    |proj g|=  1.15415D-03

At iterate   35    f=  8.12451D+00    |proj g|=  1.12578D-03

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function 

# Time Series


In [29]:
from statsmodels.tsa.stattools import adfuller

# Assuming your time series data is stored in a variable called 'ts_data'
result = adfuller(df_BTC)
print('ADF Statistic:', result[0])
print('p-value:', result[1])
print('Critical Values:', result[4])

ValueError: x is required to have ndim 1 but has ndim 2

In [26]:
import pandas as pd
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error
from math import sqrt


# Split the data into training and testing sets
train_size = int(len(df_BTC) * 0.8)
train_data, test_data = df_BTC.iloc[:train_size], df_BTC.iloc[train_size:]

# Define the SARIMA model
# Adjust the order and seasonal_order parameters based on your data and domain knowledge
order = (2, 1, 1)  # ARIMA order
seasonal_order = (1, 1, 1, 12)  # SARIMA seasonal order
model = SARIMAX(train_data['Highest Price (USD)'], order=order, seasonal_order=seasonal_order, enforce_stationarity=False)

sarima_model = model.fit()

# Predictions
start_index = len(train_data)
end_index = start_index + len(test_data) - 1
predictions = sarima_model.predict(start=start_index, end=end_index, dynamic=False)

# Calculate RMSE (Root Mean Squared Error)
mse = mean_squared_error(test_data['Highest Price (USD)'], predictions)
rmse = sqrt(mse)
print("Root Mean Squared Error (RMSE):", rmse)

# You can now use the trained SARIMA model to make predictions for future time periods
# For example, to predict the next 7 days:
# future_forecast = sarima_model.forecast(steps=7)


/Users/nikolasstahl/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/Users/nikolasstahl/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
 This problem is unconstrained.


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            6     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  8.20204D+00    |proj g|=  4.95948D-02

At iterate    5    f=  8.19963D+00    |proj g|=  3.48393D-03

At iterate   10    f=  8.19868D+00    |proj g|=  1.12245D-02

At iterate   15    f=  8.19833D+00    |proj g|=  7.90586D-04

At iterate   20    f=  8.19830D+00    |proj g|=  1.55466D-03

At iterate   25    f=  8.19665D+00    |proj g|=  8.21315D-03

At iterate   30    f=  8.18945D+00    |proj g|=  5.13126D-02

At iterate   35    f=  8.13130D+00    |proj g|=  4.33306D-02

At iterate   40    f=  8.12533D+00    |proj g|=  1.32740D-02

At iterate   45    f=  8.12157D+00    |proj g|=  7.24784D-03

At iterate   50    f=  8.12146D+00    |proj g|=  2.49267D-05

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cau

/Users/nikolasstahl/anaconda3/lib/python3.11/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


Root Mean Squared Error (RMSE): 17977.03935295564


In [27]:
future_forecast_TS = sarima_model.forecast(steps=7)
future_forecast_TS

2023-08-25    26464.953783
2023-08-26    26496.188539
2023-08-27    26467.920784
2023-08-28    26472.421535
2023-08-29    26405.706561
2023-08-30    26016.883636
2023-08-31    25971.673896
Freq: D, Name: predicted_mean, dtype: float64